In [ ]:
!pip install deeprobust==0.2.4

!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html


# 第四课 图神经网络鲁棒性

在前面课程中，我们学习了图神经网络的基础实现，并且见识了图神经网络在图相关任务上的卓越能力。但是，图神经网络存在着鲁棒性的问题：图神经网络在面对对抗攻击（adversarial attack）时表现并不好。换而言之，当攻击者（attacker）对图数据进行微小的、刻意的改动时，图神经网络的性能会大幅下降。

在本次课上，我们将使用DeepRobust工具包，来进行图神经网络上的对抗攻击与防御的实践。本节课的目标是让大家熟练掌握这个对抗攻击与防御的使用，而不涉及到具体的算法细节。学有余力的同学可以根据对应的文献链接深入学习。

## 1. DeepRobust基础

DeepRobust是由密歇根州立大学汤继良教授团队开发的针对于对抗攻击与防御的工具包，发表于顶级会议AAAI 2021。它基于PyTorch实现，包含了图神经网络领域和计算机视觉领域的常见攻击和防御的算法。该工具包在图鲁棒性领域被经常使用，其具体代码链接可见https://github.com/DSE-MSU/DeepRobust ，也欢迎大家点个收藏，star一下：）谢谢！ 

### 1.1 DeepRobust安装

对于DeepRobust的安装，我们选用如下任何一种方式：

1. 使用pip直接安装
  
  `pip install deeprobust==0.2.4`


2. 源码安装
  
  `git clone https://github.com/DSE-MSU/DeepRobust.git
cd DeepRobust
python setup.py install` 

### 1.2 利用DeepRobust加载数据集

DeepRobust工具包按照应用领域划分为graph和image两个模块。其中graph模块针对于图神经网络，这也是本次课所需要用到的。

#### 1.2.1 加载未被攻击的数据集

DeepRobust支持下载常见的标准数据集，比如Cora, Cora-ML, Citeseer, Pubmed, Polblogs, ACM, BlogCatalog, Flickr和UAI。这里我们继续以Cora数据集为例，使用`deeprobust.graph.data.Dataset`来展示加载数据集的过程。

In [ ]:
from deeprobust.graph.data import Dataset
data = Dataset(root='./', name='cora', setting='prognn') # 数据集由ProGNN提供: https://github.com/ChandlerBang/Pro-GNN
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

Loading cora dataset...
Selecting 1 largest connected components


In [ ]:
adj

<2485x2485 sparse matrix of type '<class 'numpy.float32'>'
	with 10138 stored elements in Compressed Sparse Row format>

In [ ]:
features

<2485x1433 sparse matrix of type '<class 'numpy.float32'>'
	with 45487 stored elements in Compressed Sparse Row format>

In [ ]:
print(idx_train.shape) 
print(idx_val.shape) 
print(idx_test.shape)

(247,)
(249,)
(1988,)


值得注意的是
* `deeprobust.graph.data.Dataset`存储了图的邻接矩阵、节点特征矩阵和训练/验证/测试节点。
* `deeprobust.graph.data.Dataset`接收一个setting参数，该参数可以是`nettack`、`prognn`或者`gcn`。`nettack`表示我们采用Nettack算法中使用的设置，即选取图的最大连通图，并且训练集/验证集/测试集的划分采用10%/10%/80%；`prognn`的设置同`nettack`一样，只是数据集的划分是固定的（由ProGNN提供），不随着随机种子的变化而变化；`gcn`表示我们采用GCN算法中使用的设置，即使用整张图，并且训练集由每个类别的20个节点构成，验证集/测试集是500/1000个节点。


#### 1.2.2 加载攻击后的数据集

In [ ]:
from deeprobust.graph.data import PrePtbDataset

# 加载攻击后的数据集
perturbed_data = PrePtbDataset(root='./',
                                name='cora',
                                attack_method='meta',
                                ptb_rate=0.05)
perturbed_adj = perturbed_data.adj

Loading cora dataset perturbed by 0.05 meta...


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/data/attacked_data.py:135: UserWarning: The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').
  warnings.warn("The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').")


值得注意的是：
* `deeprobust.graph.data.PrePtbDataset`提供的是被攻击后的邻接矩阵，并不涉及到对节点特征的改变。
* `attack_method`表示攻击算法，这里我们采用的是`meta`，也就是Metattack (https://arxiv.org/abs/1902.08412) ，它是一个无目标攻击，旨在对整个图进行扰动。
* `ptb_rate`表示扰动的幅度，这里我们采用的0.05，即在原图上产生5%的扰动。

### 1.3 防御模型

`deeprobust.graph.defense`中提供了多种防御模型，其中也包括了受害模型GCN。因此，我们可以直接调用`deeprobust.graph.defense.GCN`来完成多种测试。

In [ ]:
from deeprobust.graph.defense import GCN
gcn = GCN(nfeat=features.shape[1],
    nhid=16,
    nclass=labels.max().item() + 1,
    dropout=0.5, device='cuda')
gcn = gcn.to('cuda')
gcn.fit(features, adj, labels, idx_train, idx_val) # 在未被攻击的图上训练
gcn.test(idx_test)  # 测试

Test set results: loss= 0.6183 accuracy= 0.8229


0.8229376257545271

可见，GCN在未被攻击的图上的正确率是83.6%

## 2 图节点攻击

目前对图数据的对抗攻击主要集中在节点分类任务上，因此这里我们也主要讲解图节点攻击。

图节点攻击可以根据划分攻击的对象划分为有目标攻击和无目标攻击：
- 有目标攻击：在给定一小部分测试节点 (或目标节点) 的情况下，攻击者的目标是使模型对这些测试样本进行错误分类。
- 无目标攻击：攻击者的目标是通过扰动图数据以降低模型的整体性能。

### 2.1 有目标攻击

`deeprobust.graph.targeted_attack`提供了多种有目标攻击方法，详情可见该[链接](https://deeprobust.readthedocs.io/en/latest/graph/attack.html#targeted-attack-for-node-classification)。

Nettack (https://arxiv.org/abs/1805.07984) 算法是经典的有目标攻击。给定待攻击的目标节点，Nettack会生成针对于该节点的扰动，这种扰动可以是改变目标节点的边，也可以是改变目标节点的节点特征。下面根据[test_nettack.py](https://github.com/DSE-MSU/DeepRobust/blob/master/examples/graph/test_nettack.py)来具体展示Nettack的使用与测试。

那么首先我们加载数据集。

In [ ]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.targeted_attack import Nettack
data = Dataset(root='./', name='cora', setting='prognn')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test

Loading cora dataset...
Selecting 1 largest connected components


通常我们在生成对抗攻击的时候，会假定我们不知道被攻击的模型是什么。因此采用一个代理模型来生成攻击。这个代理模型一个2层的线性GCN。

In [ ]:
# 设置代理模型
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, dropout=0, with_relu=False, with_bias=False, device='cuda').to('cuda')
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)

In [ ]:
# 设置攻击模型
target_node = 0
device = 'cuda'
model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, 
                attack_features=True, device=device).to(device)
# 产生攻击
model.attack(features, adj, labels, target_node, n_perturbations=5) # n_perturbations: 改动的数量
modified_adj = model.modified_adj 
modified_features = model.modified_features 

##### Starting attack #####
##### Attack node with ID 0 using structure and feature perturbations #####
##### Attacking the node directly #####
##### Performing 5 perturbations #####
##### ...1/5 perturbations ... #####


/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../usr/local/lib/python3.7/dist-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


Edge perturbation: [   0 2116]
##### ...2/5 perturbations ... #####
Edge perturbation: [   0 2108]
##### ...3/5 perturbations ... #####
Edge perturbation: [   0 1130]
##### ...4/5 perturbations ... #####
Edge perturbation: [   0 1844]
##### ...5/5 perturbations ... #####
Edge perturbation: [   0 1084]


In [ ]:
def test_evasion():
    # target_gcn是被攻击的模型
    target_gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    target_gcn = target_gcn.to(device)
    target_gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)

    cnt = 0 # 错误分类的节点数量
    degrees = adj.sum(0).A1
    node_list = list(range(0, 10)) # 这里我们选取0-9目标节点
    num = len(node_list)

    print('=== 分别攻击并测试%s个节点 ===' % num)
    for target_node in (node_list):
        n_perturbations = int(degrees[target_node]) # 每个节点产生n_perturbations次扰动
        model = Nettack(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=True, device=device)
        model = model.to(device)
        model.attack(features, adj, labels, target_node, n_perturbations, verbose=False)
        modified_adj = model.modified_adj
        modified_features = model.modified_features

        acc = single_test(modified_adj, modified_features, target_node, gcn=target_gcn)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))

In [ ]:
def single_test(adj, features, target_node, gcn=None):
    # 对于给定的目标节点，测试给定的GCN能否将其预测正确
    output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

In [ ]:
from deeprobust.graph.utils import classification_margin
import numpy as np
import torch
device = 'cuda'
test_evasion()

=== 分别攻击并测试10个节点 ===
misclassification rate : 0.9


由此可见，10个节点中10个节点在攻击后都被错误分类了。

### 2.2 无目标攻击

`deeprobust.graph.global_attack`提供了多种无目标攻击方法（也称为全局攻击），详情可见该[链接](https://deeprobust.readthedocs.io/en/latest/graph/attack.html#global-untargeted-attack-for-node-classification)。

Metattack (https://arxiv.org/abs/1902.08412) 是经典的无目标攻击算法。攻击者的目标是通过扰动图数据以降低模型的整体性能。

In [ ]:
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 设置代理模型
device = 'cuda'
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
            nhid=16, dropout=0, with_relu=False, with_bias=False, device=device).to(device)
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=30)

In [ ]:
# 设置攻击模型
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
    attack_structure=True, attack_features=False, device=device, lambda_=0).to(device)
# 产生攻击
model.attack(features, adj, labels, idx_train, idx_unlabeled, n_perturbations=30, ll_constraint=False)
modified_adj = model.modified_adj

Perturbing graph:   0%|          | 0/30 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5377652645111084
GCN acc on unlabled data: 0.835493965131873
attack loss: 0.3010116517543793


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/utils.py:541: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  rows = index // array_shape[1]
Perturbing graph:   3%|▎         | 1/30 [00:16<07:49, 16.20s/it]

GCN loss on unlabled data: 0.5307962894439697
GCN acc on unlabled data: 0.8319177469825659
attack loss: 0.3226845860481262


Perturbing graph:   7%|▋         | 2/30 [00:32<07:28, 16.01s/it]

...

Perturbing graph:  93%|█████████▎| 28/30 [07:30<00:32, 16.10s/it]

GCN loss on unlabled data: 0.6233600974082947
GCN acc on unlabled data: 0.8082253017434063
attack loss: 0.48184916377067566


Perturbing graph:  97%|█████████▋| 29/30 [07:46<00:16, 16.12s/it]

GCN loss on unlabled data: 0.5930318236351013
GCN acc on unlabled data: 0.8287885561019223
attack loss: 0.4545987844467163


Perturbing graph: 100%|██████████| 30/30 [08:03<00:00, 16.10s/it]


可以看到GCN的准确率在不断下降，如果我们增大`n_perturbation`，即产生更多的扰动，我们会发现GCN的准确率降得更多。

## 3. 图防御

`deeprobust.graph.defense`既提供了受害模型，也提供了防御模型。

### 3.1 受害模型-GCN

* GCN在未攻击的数据集上的准确率

In [ ]:
from deeprobust.graph.defense import GCN
gcn = GCN(nfeat=features.shape[1],
    nhid=16,
    nclass=labels.max().item() + 1,
    dropout=0.5, device='cuda')
gcn = gcn.to('cuda')
gcn.fit(features, adj, labels, idx_train, idx_val) # train on clean graph with earlystopping
gcn.test(idx_test)

Test set results: loss= 0.5667 accuracy= 0.8335


0.8335010060362174

* GCN在攻击后的数据集上的准确率

In [ ]:
from deeprobust.graph.data import PrePtbDataset

# 加载攻击后的数据集
perturbed_data = PrePtbDataset(root='./',
                               name='cora',
                               attack_method='meta',
                               ptb_rate=0.15)
perturbed_adj = perturbed_data.adj

Dowloading from https://raw.githubusercontent.com/ChandlerBang/Pro-GNN/master/meta/cora_meta_adj_0.15.npz to ./cora_meta_adj_0.15.npz
Loading cora dataset perturbed by 0.15 meta...


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/data/attacked_data.py:135: UserWarning: The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').
  warnings.warn("The pre-attacked graph is perturbed under the data splits provided by ProGNN. So if you are going to verify the attacking performance, you should use the same data splits  (setting='prognn').")


In [ ]:
from deeprobust.graph.defense import GCN
gcn = GCN(nfeat=features.shape[1],
    nhid=16,
    nclass=labels.max().item() + 1,
    dropout=0.5, device='cuda')
gcn = gcn.to('cuda')
gcn.fit(features, perturbed_adj, labels, idx_train, idx_val) # 在攻击后的图上训练
gcn.test(idx_test) # 测试

Test set results: loss= 1.0685 accuracy= 0.6640


0.6639839034205232

### 3.2 防御模型

`deeprobust.graph.defense`提供了多种防御模型，见https://deeprobust.readthedocs.io/en/latest/graph/defense.html 。这里我们介绍三种防御模型：GCN-Jaccard, GCN-SVD和SimP-GCN。

#### 3.2.1 GCN-Jaccard

GCN-Jaccard: Topology Attack and Defense for Graph Neural Networks: An Optimization Perspective. IJCAI 2019. [[link]](https://arxiv.org/abs/1906.04214)

这是一个preprocessing的方法，设定一个阈值，如果一条边连接的节点的特征相似度小于这个阈值，就将这条边从原图中去除掉。

In [ ]:
from deeprobust.graph.defense import GCNJaccard
device = 'cuda'
model = GCNJaccard(nfeat=features.shape[1],
          nhid=16,
          nclass=labels.max().item() + 1,
          dropout=0.5, device=device).to(device)
model.fit(features, perturbed_adj, labels, idx_train, idx_val, threshold=0.03) # threshold: 阈值
model.test(idx_test)

removed 1518 edges in the original graph
=== training gcn model ===
Epoch 0, training loss: 2.058633804321289
Epoch 10, training loss: 1.2342873811721802
Epoch 20, training loss: 0.7238360643386841
Epoch 30, training loss: 0.3785979747772217
Epoch 40, training loss: 0.23777393996715546
Epoch 50, training loss: 0.15310007333755493
Epoch 60, training loss: 0.13636794686317444
Epoch 70, training loss: 0.12522289156913757
Epoch 80, training loss: 0.12822803854942322
Epoch 90, training loss: 0.092674620449543
Epoch 100, training loss: 0.08053364604711533
Epoch 110, training loss: 0.08960019797086716
Epoch 120, training loss: 0.08758901804685593
Epoch 130, training loss: 0.08124297857284546
Epoch 140, training loss: 0.09362863004207611
Epoch 150, training loss: 0.07094777375459671
Epoch 160, training loss: 0.07394914329051971
Epoch 170, training loss: 0.07121925801038742
Epoch 180, training loss: 0.07081296294927597
Epoch 190, training loss: 0.08245762437582016
=== picking the best model acc

0.7535211267605634

#### 3.2.2 GCN-SVD

GCN-SVD: 
All You Need Is Low (Rank): Defending Against Adversarial Attacks on Graphs. WSDM 2020. [[link]](https://dl.acm.org/doi/10.1145/3336191.3371789)

这是一个preprocessing的方法，对图的邻接矩阵进行低秩重建，重建后的矩阵作为新的邻接矩阵并用于之后的训练。

In [ ]:
from deeprobust.graph.defense import GCNSVD
device = 'cuda'
model = GCNSVD(nfeat=features.shape[1],
          nhid=16,
          nclass=labels.max().item() + 1,
          dropout=0.5, device=device).to(device)
model.fit(features, perturbed_adj, labels, idx_train, idx_val, k=100) # 希望新的邻接矩阵的秩(rank)是100
model.test(idx_test)

=== GCN-SVD: rank=100 ===
rank_after = 100
=== training gcn model ===
Epoch 0, training loss: 1.9718362092971802
Epoch 10, training loss: 1.4565094709396362
Epoch 20, training loss: 0.9874127507209778
Epoch 30, training loss: 0.7903996109962463
Epoch 40, training loss: 0.6287311911582947
Epoch 50, training loss: 0.5186724662780762
Epoch 60, training loss: 0.5083103179931641
Epoch 70, training loss: 0.4508126676082611
Epoch 80, training loss: 0.4240659773349762
Epoch 90, training loss: 0.3856407701969147
Epoch 100, training loss: 0.3562161326408386
Epoch 110, training loss: 0.366477906703949
Epoch 120, training loss: 0.3479040563106537
Epoch 130, training loss: 0.3532334268093109
Epoch 140, training loss: 0.32573994994163513
Epoch 150, training loss: 0.3254378139972687
Epoch 160, training loss: 0.3263048231601715
Epoch 170, training loss: 0.2932012975215912
Epoch 180, training loss: 0.3258059620857239
Epoch 190, training loss: 0.3068508803844452
=== picking the best model according to t

0.6996981891348089

#### 3.2.3 SimP-GCN

SimP-GCN: 
Node Similarity Preserving Graph Convolutional Networks. WSDM 2021. [[link]](https://arxiv.org/abs/2011.09643)

SimP-GCN是一个end-to-end的算法，其核心思想是自适应的聚合过程，即有选择性地从邻居聚合信息。

In [ ]:
from deeprobust.graph.defense import SimPGCN
device = 'cuda'
model = SimPGCN(nnodes=features.shape[0], nfeat=features.shape[1],
                nhid=16, nclass=labels.max()+1, device=device, 
                lambda_=0.1, gamma=0.01).to(device)
model.fit(features, perturbed_adj, labels, idx_train, idx_val, train_iters=200, verbose=True)
model.test(idx_test)

=== training gcn model ===
loading saved_knn/cosine_sims_(2485, 1433).npy
number of sampled: 21982
Epoch 0, training loss: 1.9385247230529785
Epoch 10, training loss: 1.2635031938552856
Epoch 20, training loss: 0.7140966653823853
Epoch 30, training loss: 0.45076146721839905
Epoch 40, training loss: 0.2526361346244812
Epoch 50, training loss: 0.16213729977607727
Epoch 60, training loss: 0.11364463716745377
Epoch 70, training loss: 0.11805734783411026
Epoch 80, training loss: 0.10647794604301453
Epoch 90, training loss: 0.07202724367380142
Epoch 100, training loss: 0.06708060950040817
Epoch 110, training loss: 0.06571284681558609
Epoch 120, training loss: 0.054884880781173706
Epoch 130, training loss: 0.06350597739219666
Epoch 140, training loss: 0.04640984907746315
Epoch 150, training loss: 0.06162116676568985
Epoch 160, training loss: 0.06910757720470428
Epoch 170, training loss: 0.056473277509212494
Epoch 180, training loss: 0.05187642574310303
Epoch 190, training loss: 0.037898067384

0.7751509054325956